In [352]:
import pandas as pd
import time
import plotly.graph_objects as go 
import plotly.express as px
import numpy as np

def JsonParser(data):
    import json
    js = json.loads(data)
    return js


df_list = pd.read_csv('listings.csv', converters={'amenities':JsonParser})

df_compare = pd.DataFrame(columns=['Nazwa', 'Czas', 'F. Straty', 'RMSE'])

C:\Users\manar\AppData\Local\Temp\ipykernel_16432\2050797132.py:13: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = pd.read_csv('listings.csv', converters={'amenities':JsonParser})


In [353]:
df_list.shape

(39881, 75)

In [354]:
df_list['source'].unique()

array(['city scrape', 'previous scrape'], dtype=object)

In [355]:
df_list['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Hotel room', 'Shared room'],
      dtype=object)

In [356]:
df_list['price']

0        $299.00
1        $175.00
2         $60.00
3        $425.00
4        $275.00
          ...   
39876    $157.00
39877    $108.00
39878    $140.00
39879    $400.00
39880     $40.00
Name: price, Length: 39881, dtype: object

In [357]:
df_list.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [358]:
columns_not_incluted = ['id','listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified','latitude', 'longitude',
       'calendar_last_scraped','first_review',
       'last_review','license', 
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'
       ]

In [359]:
df_list = df_list.drop(columns_not_incluted, axis=1)
df_list.head(5)

,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,...,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,"Brooklyn , New York, United States",Kensington,Brooklyn,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,1.0,...,0,4.89,4.88,5.00,5.00,5.00,4.75,4.88,f,0.11
1,"New York, United States",Midtown,Manhattan,Entire rental unit,Entire home/apt,1,NaN,1 bath,NaN,1.0,...,0,4.68,4.73,4.63,4.77,4.80,4.81,4.40,f,0.31
2,NaN,Bedford-Stuyvesant,Brooklyn,Private room in rental unit,Private room,2,NaN,NaN,1.0,1.0,...,0,4.52,4.22,4.09,4.91,4.91,4.47,4.52,f,0.31
3,NaN,Ridgewood,Queens,Entire townhouse,Entire home/apt,16,NaN,2.5 baths,5.0,10.0,...,0,4.42,4.64,4.36,4.82,5.00,4.82,4.55,f,0.10
4,NaN,Sunset Park,Brooklyn,Entire rental unit,Entire home/apt,4,NaN,1.5 baths,2.0,2.0,...,1,5.00,5.00,5.00,5.00,5.00,4.67,5.00,f,0.03


In [360]:
df_list['bathrooms_text'].unique(), df_list['bathrooms_text'].isna().sum()

(array(['1 shared bath', '1 bath', nan, '2.5 baths', '1.5 baths',
        '1 private bath', '1.5 shared baths', '2 baths', '2 shared baths',
        'Shared half-bath', '3 baths', 'Half-bath', '0 baths',
        '0 shared baths', 'Private half-bath', '4 baths', '4.5 baths',
        '5.5 baths', '2.5 shared baths', '3.5 baths', '5 baths',
        '3 shared baths', '4 shared baths', '6 baths', '3.5 shared baths',
        '4.5 shared baths', '7.5 baths', '6.5 baths', '7 baths',
        '6 shared baths', '9.5 baths'], dtype=object),
 84)

In [361]:
df_list['beds'].unique()

array([ 1., 10.,  2.,  5.,  3.,  8.,  4.,  6., nan,  9.,  7., 18., 12.,
       16., 11., 21., 13., 24., 17., 14.])

In [362]:
df_list.isna().sum()[df_list.isna().sum() > 0]

neighbourhood                  16414
bathrooms                      39881
bathrooms_text                    84
bedrooms                        3783
beds                             884
calendar_updated               39881
review_scores_rating            8362
review_scores_accuracy          8825
review_scores_cleanliness       8815
review_scores_checkin           8829
review_scores_communication     8820
review_scores_location          8832
review_scores_value             8832
reviews_per_month               8362
dtype: int64

In [363]:
columns_with_nulls = ['neighbourhood' ,'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated']

In [364]:
df_list = df_list.drop(columns_with_nulls, axis=1)

In [365]:
#df_list = df_list[df_list['reviews_per_month'] > 0]
df_list = df_list.dropna()
df_list.isna().sum()[df_list.isna().sum() > 0]
df_list.shape, df_list.columns

((27578, 26),
 Index(['neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates',
        'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
        'has_availability', 'availability_30', 'availability_60',
        'availability_90', 'availability_365', 'number_of_reviews',
        'number_of_reviews_ltm', 'number_of_reviews_l30d',
        'review_scores_rating', 'review_scores_accuracy',
        'review_scores_cleanliness', 'review_scores_checkin',
        'review_scores_communication', 'review_scores_location',
        'review_scores_value', 'instant_bookable', 'reviews_per_month'],
       dtype='object'))

In [366]:
df_list.dtypes

neighbourhood_cleansed          object
property_type                   object
room_type                       object
accommodates                     int64
bathrooms_text                  object
bedrooms                       float64
beds                           float64
amenities                       object
price                           object
has_availability                object
availability_30                  int64
availability_60                  int64
availability_90                  int64
availability_365                 int64
number_of_reviews                int64
number_of_reviews_ltm            int64
number_of_reviews_l30d           int64
review_scores_rating           float64
review_scores_accuracy         float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
review_scores_value            float64
instant_bookable                object
reviews_per_month        

In [367]:
not_numeric = ['neighbourhood_cleansed', 'property_type', 'room_type', 'bathrooms_text', 'amenities', 'has_availability', 'instant_bookable']
binary_column = ['has_availability', 'instant_bookable']
not_handled = ['bathrooms_text']

In [368]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [369]:
encoder = OrdinalEncoder(categories=[['f','t']])
df_list['has_availability'] = encoder.fit_transform(df_list[['has_availability']])
df_list['instant_bookable'] = encoder.fit_transform(df_list[['instant_bookable']])
df_list['has_availability'] = df_list['has_availability'].astype('int')
df_list['instant_bookable'] = df_list['instant_bookable'].astype('int')

In [370]:
from re import sub
def remove_dollar_sign(price:str):
    return price.replace('$','').replace(',','')

def remove_quotes(value:str):
    return value.replace('"','')

def remove_weird_signs(row):
    row = row.encode('ascii', 'ignore').strip().decode('unicode_escape').replace('"','').replace('[','').replace(']','')
    row = ''.join([i if ord(i) < 128 else '' for i in row])
    row_as_list = row.split(',')
    row_as_list = [x.strip() for x in row_as_list]
    return ','.join(row_as_list)

def clear_amenities(row):
    to_clear = ['wifi','childrens','tv','sound','parking','refrigerator','dryer','washer','clothing','stove','oven','pool','gym','breakfast','garage','sauna','garden',
    'heating','hot tub','console','conditioning','chair','crib','housekeeping','free carport on premises','kitchen', 'ev charger', 'pillows', ' bar',  'fitness','restaurant']
    row = [x for x in row if x not in 'chair']
    cosmetics = ['conditioner', 'soap', 'shampoo', 'toiletries']
    sound = ['sound', 'speaker']
    row = ['cosmetics' if any(cos in x for cos in cosmetics) else x.lower() for x in row]
    row = ['sound' if any(cos in x for cos in sound) else x.lower() for x in row]
    for item in to_clear:
        row = [f'{item}' if f'{item}' in x.lower() else x.lower() for x in row]
    return row

In [371]:
df_list['price'] =  df_list['price'].apply(remove_dollar_sign).astype('float32')
df_list['amenities'] = df_list['amenities'].apply(clear_amenities)

In [372]:
df_list['property_type'].unique()

array(['Private room in rental unit', 'Entire townhouse',
       'Entire rental unit', 'Room in boutique hotel',
       'Private room in bed and breakfast', 'Private room in townhouse',
       'Private room in home', 'Private room in condo',
       'Private room in loft', 'Entire guesthouse', 'Entire guest suite',
       'Entire loft', 'Private room in guest suite',
       'Entire serviced apartment', 'Entire condo', 'Entire home',
       'Shared room in loft', 'Private room in houseboat',
       'Shared room in rental unit', 'Shared room in home',
       'Private room', 'Entire place', 'Private room in guesthouse',
       'Room in aparthotel', 'Entire cottage',
       'Private room in serviced apartment', 'Boat', 'Cave',
       'Room in serviced apartment', 'Tiny home', 'Shared room in condo',
       'Floor', 'Private room in hostel',
       'Private room in casa particular', 'Room in hotel',
       'Shared room in townhouse', 'Private room in bungalow',
       'Houseboat', 'Entire bu

In [373]:
def clean_property_type(row):
    property_type_remove = ['entire', 'private', 'room','tiny', 'in',  'shared']
    row_list = row.split()
    row_list = [x for x in row_list if x.lower() not in property_type_remove]
    return ' '.join(row_list).lower()

In [374]:
df_list['property_type'] = df_list['property_type'].apply(clean_property_type)

In [375]:
df_list['property_type'].unique()

array(['rental unit', 'townhouse', 'boutique hotel', 'bed and breakfast',
       'home', 'condo', 'loft', 'guesthouse', 'guest suite',
       'serviced apartment', 'houseboat', '', 'place', 'aparthotel',
       'cottage', 'boat', 'cave', 'floor', 'hostel', 'casa particular',
       'hotel', 'bungalow', 'resort', 'villa', 'tent', 'earthen home',
       'lighthouse', 'barn', 'vacation home', 'camper/rv', 'minsu',
       'tower'], dtype=object)

In [376]:
#need to be saperate to number and isbathshared
df_list['bathrooms_text'].unique()

array(['1 shared bath', '2.5 baths', '1.5 baths', '1 private bath',
       '1 bath', '1.5 shared baths', '2 baths', '2 shared baths',
       'Shared half-bath', '3 baths', 'Half-bath', '0 shared baths',
       'Private half-bath', '4 baths', '4.5 baths', '5.5 baths',
       '2.5 shared baths', '3.5 baths', '0 baths', '3 shared baths',
       '5 baths', '4 shared baths', '6 baths', '4.5 shared baths',
       '6.5 baths', '7 baths', '6 shared baths'], dtype=object)

In [377]:
def set_bath_type(bath:str) -> str:
    if('shared'):
        return 1
    else:
        return 0

In [378]:
def set_baths_number(bath:str) -> float:
    import re
    bath = bath.lower().replace('half','0.5')
    bath = re.findall(r"[-+]?(?:\d*\.*\d+)", bath)[0]
    return float(bath)

In [379]:
df_list['bath_number'] = df_list['bathrooms_text'].apply(set_baths_number).astype('float32')
df_list['bath_is_shared'] = df_list['bathrooms_text'].apply(set_bath_type).astype('int')

In [380]:
df_list.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,has_availability,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,bath_number,bath_is_shared
0,Kensington,rental unit,Private room,2,1 shared bath,1.0,1.0,"[first aid kit, microwave, stove, coffee maker...",299.0,1,...,4.88,5.00,5.00,5.00,4.75,4.88,0,0.11,1.0,1
3,Ridgewood,townhouse,Entire home/apt,16,2.5 baths,5.0,10.0,"[dryer, essentials, carbon monoxide alarm, iro...",425.0,1,...,4.64,4.36,4.82,5.00,4.82,4.55,0,0.10,2.5,1
4,Sunset Park,rental unit,Entire home/apt,4,1.5 baths,2.0,2.0,"[dryer, tv, refrigerator, bbq grill, garden, h...",275.0,1,...,5.00,5.00,5.00,5.00,4.67,5.00,0,0.03,1.5,1
6,Greenpoint,boutique hotel,Hotel room,2,1 private bath,1.0,1.0,"[first aid kit, coffee maker, gym, long term s...",308.0,1,...,4.98,4.95,4.98,4.88,4.46,4.78,0,0.30,1.0,1
7,Midtown,rental unit,Private room,2,1 bath,1.0,1.0,"[cosmetics, microwave, laundromat nearby, mini...",68.0,1,...,4.24,3.75,4.66,4.46,4.88,4.40,0,3.39,1.0,1


In [381]:
df_list = df_list.drop(columns=not_handled)
df_list.shape

(27578, 27)

In [382]:
df_list_without_dummies = df_list.copy()

In [383]:
df_list["price"].quantile(0.99)

995.0

In [384]:
df_list["availability_365"].quantile(0.97)

361.0

In [385]:
df_list = df_list[df_list['availability_365'] != 365]
df_list["price"].quantile(0.99)

979.130000000001

In [386]:
df_list = df_list[(df_list['price']) < (df_list["price"].quantile(0.99))]

In [387]:
amenities_dummies = pd.get_dummies(df_list['amenities'].apply(pd.Series).stack()).groupby(level=0).sum()
df_list = df_list.join(amenities_dummies)
df_list = df_list.drop(columns=['amenities'])
df_list =  pd.get_dummies(df_list)

df_list.shape

C:\Users\manar\AppData\Local\Temp\ipykernel_16432\2584880822.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  amenities_dummies = pd.get_dummies(df_list['amenities'].apply(pd.Series).stack()).groupby(level=0).sum()


(26916, 429)

In [388]:
df_list.columns

Index(['accommodates', 'bedrooms', 'beds', 'price', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews',
       ...
       'property_type_serviced apartment', 'property_type_tent',
       'property_type_tower', 'property_type_townhouse',
       'property_type_vacation home', 'property_type_villa',
       'room_type_Entire home/apt', 'room_type_Hotel room',
       'room_type_Private room', 'room_type_Shared room'],
      dtype='object', length=429)

In [389]:
df_list.shape

(26916, 429)

In [390]:
df_list = df_list.dropna()
df_list.shape

(26915, 429)

In [391]:
df_list_without_dummies.to_csv('df_without_dummies.csv')
df_list.to_csv('df_with_dummies.csv')

In [392]:
df_list['price'].describe()

count    26915.000000
mean       165.651230
std        130.634628
min         10.000000
25%         78.000000
50%        126.000000
75%        200.000000
max        979.000000
Name: price, dtype: float64